# Plan Semilla 3.0 - Artificial Intelligence Module

# State Space Search

In this challenge you will implement some search algorithms for navigation problems.

Your task is to create an autonomous vehicle that goes from point A to point B. You will be given some grid configurations that will simulate aerial images.

The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:


In [1]:
import numpy as np

# symbols with meanings and costs
print('Symbol', '   Meaning       ', 'Cost')
print('🛣', '       Street        ', 1)
print('🌲', '       Forest        ', 10)
print('🚸', '       School street ', 2)
print('🚦', '       Traffic light ', 3)
print('🚔', '       Police        ', 4)
print('🚘', '       Traffic       ', 6)
print('🛑', '       Stop sign     ', 5)
print('🚧', '       Construction  ', 7)
print('🏢', '       Buildings     ', 15)

# dictionary: Symbols with weights (costs)
COSTS = {'🛣': 1, '🌲': 10, '🚸': 2, '🚦': 3, '🚔': 4, '🚘': 6, '🛑': 5, '🚧': 7, '🏢': 15}

Symbol    Meaning        Cost
🛣        Street         1
🌲        Forest         10
🚸        School street  2
🚦        Traffic light  3
🚔        Police         4
🚘        Traffic        6
🛑        Stop sign      5
🚧        Construction   7
🏢        Buildings      15


In [2]:

test_map_1 = np.array([['🛣', '🛣', '🛣', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🛣', '🛣', '🛣'],
                       ['🌲', '🛣', '🛣', '🛣']])

map1 = np.array([['🛣', '🚸', '🛣', '🛣', '🛣', '🛣', '🛣', '🛑', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🚔', '🛣'],
                 ['🛣', '🌲', '🌲', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🌲', '🌲', '🛣', '🚘', '🛣'],
                 ['🛣', '🛑', '🛣', '🚸', '🛣', '🌲', '🛣', '🛣', '🏢', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚦'],
                 ['🌲', '🚘', '🚘', '🛣', '🛣', '🛣', '🚘', '🛣', '🏢', '🛣', '🛣', '🌲', '🛣', '🛑', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🛣', '🚸'],
                 ['🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛑', '🛣', '🛣', '🛣', '🚘', '🛣', '🛣', '🚸', '🛣', '🛣'],
                 ['🚦', '🛣', '🏢', '🚔', '🛣', '🛣', '🏢', '🚧', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚔'],
                 ['🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🏢', '🏢', '🏢', '🚸', '🛣', '🛑', '🚘', '🚔', '🛣', '🛣'],
                 ['🛣', '🛑', '🛣', '🛣', '🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🚘'],
                 ['🛣', '🛣', '🛣', '🌲', '🌲', '🚦', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚧', '🚘'],
                 ['🚘', '🚸', '🛣', '🌲', '🌲', '🛣', '🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🚦', '🚘', '🛣', '🛣'],
                 ['🚘', '🛣', '🚧', '🛣', '🛣', '🛣', '🛑', '🛣', '🚘', '🚘', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣']])

In [3]:


class Agent:

    def __init__(self, world, costs, initial_state, goal_state):

        self.moves = np.array([[0,1], [0,-1], [-1,0], [1,0]])
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.current_state = initial_state
        self.world = world
        self.costs = costs

    def possible_moves(self):
        """
         input: 
        
                Current state of the world
                Set of moves
        """

        succ_states, succ_actions, succ_costs = [],[],[]

        for move in self.moves:

            # check that the next state is in the range of the world
            #print(self.current_state)
            if 0 <= self.current_state[0] + move[0] < self.world.shape[0] and 0 <= self.current_state[1] + move[1] < self.world.shape[1]:

                # move and assign new_state to the position after moving
                new_state = [self.current_state[0] + move[0], self.current_state[1] + move[1]]

                action = [move[0], move[1]]

                cost = self.costs[self.world[new_state[0], new_state[1]]]

                succ_states.append(new_state)
                succ_actions.append(action)
                succ_costs.append(cost)

        """
         output: 
                List of possible successor states, actions, and costs.
        """
        return succ_states, succ_actions, succ_costs


    def set_current_state(self, nex_state):
        
        """
        output:
                new state
        """
        self.current_state = nex_state



class Node:

    def __init__(self, state, parent=None, action=None, cost=None):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        #self.cost = cost

        self.depth = 0
        self.path_cost = 0

        if parent:
            self.path_cost = parent.path_cost + cost
            self.depth = parent.depth + 1


    def successors(self, agent):

        succ_states, succ_actions, succ_costs = agent.possible_moves()
        children = []

        for i in range(len(succ_states)):

            children.append(Node(succ_states[i], self, succ_actions[i], succ_costs[i]))
        return  children

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []

        # Team init
        while node:
            path_back.append(node)
            node = node.parent
        # Team final

        return list(reversed(path_back))


def depth_first_graph_search(agent):

    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    """
    This initializes a frontier list with a single node representing the initial state of the agent. 
    This is a stack, so nodes will be added and removed from the end of the list.
    """
    frontier = [(Node(agent.initial_state))]  # Stack



    # Team init


    # This initializes an empty set to keep track of the explored states
    explored = set()

    # This starts a loop that continues until the frontier is not empty.
    while frontier:

        # This pops the last element from the frontier list, 
        # which represents the deepest unexplored node in the search tree.

        node = frontier.pop()

        # This checks if the state of the node is the goal state, 
        # and if so, returns the path from the initial state to the
        # goal state using the node.path() method.
        
        if node.state == agent.goal_state:
            return node.path()  # return the path to the goal node
        
        # This adds the state of the node to the explored set.
        explored.add(node.state)
        
        # This loops through the possible successor states, 
        # actions, and costs using the agent.possible_moves() method.

        for succ_state, succ_action, succ_cost in agent.possible_moves():
            # This checks if the successor state has not been explored before.
            if succ_state not in explored:
                # This creates a new Node object representing the successor state, 
                # with the node as the parent, the succ_action as the action taken 
                # to reach the state, and succ_cost as the cost of the action.
                child_node = Node(succ_state, node, succ_action, succ_cost)
                # This adds the child_node to the end of the frontier list.
                frontier.append(child_node)
    # If the frontier becomes empty, the search has failed and None is returned.
    return None

    # Team final

def breadth_first_graph_search(agent):
    #TODO
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Queue

    explored = []
    while frontier:
        pass
    return None



def uniform_cost_search(agent):
    #TODO
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Stack

    explored = []
    while frontier:
        pass

    return None


def order_frontier(frontier):

    costs = []
    new_list = []

    for i in range(len(frontier)):

        costs.append(frontier[i].path_cost)

    costs = np.array(costs)

    indexes = np.argsort(costs)

    for i in range(len(frontier)):

        new_list.append(frontier[indexes[i]])

    return list(reversed(new_list))



Ada = Agent(map1, COSTS, [0,0], [12,15])
g_node = uniform_cost_search(Ada)

print(g_node.state)

p = g_node.path()

moves = {'01':'⏩','0-1':'⏪','-10':'⏫', '10':'⏬'}

map_copy = map1.copy()


for i in range(len(p)-1):

    ac = p[i]
    n = p[i+1]

    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]


for i in range(map_copy.shape[0]):
    print(map_copy[i,:])

# Pruebas

In [14]:


class Agent:

    def __init__(self, world, costs, initial_state, goal_state):
        
        # Hypothesis 1

        # [0,1] --> 1 UP
        # [0,-1] --> 1 Down
        # [-1,0] --> 1 Left
        # [1,0] --> 1 right

        # Hypothesis 2

        # [0,1] --> 1 right column
        # [0,-1] --> 1 left column
        # [-1,0] --> 1 up row
        # [1,0] --> 1 down row


        self.moves = np.array([[0,1], [0,-1], [-1,0], [1,0]])
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.current_state = initial_state
        self.world = world
        self.costs = costs

    def possible_moves(self):
        """
         input: 
        
                Current state of the world
                Set of moves
        """

        succ_states, succ_actions, succ_costs = [],[],[]

        for move in self.moves:

            # check that the next state is in the range of the world
            #print(self.current_state)

            # shape[0] --> rows
            # shape[1] --> columns

            # self.current_state[0] is x axis 
            # self.world.shape[0] is last row (X)

            # 0 <= self.current_state[0] + move[0] < self.world.shape[0] is X boundaries
            # 0 <= self.current_state[1] + move[1] < self.world.shape[1] is Y boundaries

            if 0 <= self.current_state[0] + move[0] < self.world.shape[0] and 0 <= self.current_state[1] + move[1] < self.world.shape[1]:
                

                # move and assign new_state to the position after moving
                new_state = [self.current_state[0] + move[0], self.current_state[1] + move[1]]
                print("new_state: ",new_state)

                action = [move[0], move[1]]
                print("action: ",action)

                cost = self.costs[self.world[new_state[0], new_state[1]]]
                print("cost: ",cost)

                succ_states.append(new_state)
                succ_actions.append(action)
                succ_costs.append(cost)

        """
         output: 
                List of possible successor states, actions, and costs.
        """

        return succ_states, succ_actions, succ_costs


    def set_current_state(self, nex_state):
        
        """
        output:
                new state
        """
        self.current_state = nex_state



In [20]:

class Node:

    def __init__(self, state, parent=None, action=None, cost=None):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        #self.cost = cost

        self.depth = 0
        self.path_cost = 0

        if parent:
            self.path_cost = parent.path_cost + cost
            self.depth = parent.depth + 1


    def successors(self, agent):

        succ_states, succ_actions, succ_costs = agent.possible_moves()
        children = []

        for i in range(len(succ_states)):

            children.append(Node(succ_states[i], self, succ_actions[i], succ_costs[i]))
        return  children

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []

        # Team init
        while node:
            path_back.append(node)
            node = node.parent
        # Team final

        return list(reversed(path_back))



'\n\ndef depth_first_graph_search(agent):\n\n    #Search the deepest nodes in the search tree first.\n    #Search through the successors of a problem to find a goal.\n    #The argument frontier should be an empty queue.\n    #Does not get trapped by loops.\n    #If two paths reach a state, only use the first one.\n\n    #This initializes a frontier list with a single node representing the initial state of the agent. \n    #This is a stack, so nodes will be added and removed from the end of the list.\n\n    frontier = [(Node(agent.initial_state))]  # Stack\n\n    # Team init\n\n\n    # This initializes an empty set to keep track of the explored states\n    explored = []\n\n    # This starts a loop that continues until the frontier is not empty.\n    while frontier:\n\n        # This pops the last element from the frontier list, \n        # which represents the deepest unexplored node in the search tree.\n\n        node = frontier.pop()\n\n        # This checks if the state of the node i

In [43]:
def depth_first_graph_search(agent):
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Stack

    explored = []

    while frontier:
        print(frontier)
        if len(frontier)==1:
            print("frontier[0].state: ", frontier[0].state)
            print("frontier[0].parent: ", frontier[0].parent)
            print("frontier[0].action: ", frontier[0].action)
            print("\n")
        if len(frontier)==2:
            print("frontier[1].state: ", frontier[1].state)
            print("frontier[1].parent: ", frontier[1].parent)
            print("frontier[1].action: ", frontier[1].action)

        node = frontier.pop()

        if node.state == agent.goal_state:
            return node.path()

        explored.append(node.state)

        succ_states, succ_actions, succ_costs = agent.possible_moves()

        for i in range(len(succ_states)):
            succ_state, succ_action, succ_cost = succ_states[i], succ_actions[i], succ_costs[i]

            if not np.any([np.array_equal(succ_state, state) for state in explored]):
                child_node = Node(succ_state, node, succ_action, succ_cost)
                frontier.append(child_node)
        print("Frontier: ", frontier)
    
    
    return None


In [44]:
def breadth_first_graph_search(agent):
    #TODO
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Queue

    explored = []
    while frontier:
        pass
    return None

In [45]:
import numpy as np

world = np.array([[0, 1],
                  [2, 3]])

costs = {0: 10, 1: 1, 2: 5, 3:100}

initial_state = (0, 0)
goal_state = (1,1)

agent1 = Agent(world, costs, initial_state, goal_state)

"""
print(agent1.world)
print(agent1.costs)
print(agent1.initial_state)
print(agent1.goal_state)
print(agent1.moves)
print(agent1.possible_moves())
"""

path = depth_first_graph_search(agent1)
print(path)



frontier[0].state:  (0, 0)
frontier[0].parent:  None
frontier[0].action:  None



New data


self.current_state[0]:  0
move[0]:  0
self.world.shape[0]:  2
self.current_state[1]:  0
move[1]:  1
self.world.shape[1]:  2
move:  [0 1]
new_state:  [0, 1]
action:  [0, 1]
cost:  1

New data


self.current_state[0]:  0
move[0]:  1
self.world.shape[0]:  2
self.current_state[1]:  0
move[1]:  0
self.world.shape[1]:  2
move:  [1 0]
new_state:  [1, 0]
action:  [1, 0]
cost:  5
parent.path_cost:  0
cost:  1
parent.path_cost:  0
cost:  5
Frontier:  [<__main__.Node object at 0x0000020B14B06A30>, <__main__.Node object at 0x0000020B05E99640>]
[<__main__.Node object at 0x0000020B14B06A30>, <__main__.Node object at 0x0000020B05E99640>]
frontier[1].state:  [1, 0]
frontier[1].parent:  <__main__.Node object at 0x0000020B14B53E20>
frontier[1].action:  [1, 0]

New data


self.current_state[0]:  0
move[0]:  0
self.world.shape[0]:  2
self.current_state[1]:  0
move[1]:  1
self.world.shape[1]:  2
move:  [0 1]
new_st